In [ ]:
# Testuojama, bandant suprasti, kaip kuriamos partijos (batches) su tf.data.dataset 
# ir kaip apskaičiuoti MEAN ir STD naudojant tf.data.Dataset
# tf.data.Dataset objektas sukūriamas darbui su csv failu

import tensorflow as tf
import pandas as pd
import numpy as np
import time
import sys, os, json
from pathlib import Path

# Funkcija csv failo eilučių skaičiau suradimui
def count_lines_enumrate(file_name):
    fp = open(file_name,'r')
    for line_count, line in enumerate(fp):
        pass
    return line_count

In [ ]:
# Testuojama, bandant suprasti, kaip kuriamos partijos (batches) su tf.data.dataset 
# Sukūriame tf.data.dataset klasės objektą duomenų skaitymui partijomis iš csv failo
# Naudojamas train_features.csv, sukūrtas su 8_1_zive_create_feature_sets_cnn_vu_v1_micro.ipynb

        # Susikuriame failą bandymams su nauju stulpeliu 'nr' 

filepath = "train_features.csv"
print()
print(f'\nPožymių masyvas iš:  {filepath}')

# Eilučių skaičius csv masyve
number_of_rows = count_lines_enumrate(filepath)
print(f'\nEilučių skaičius csv masyve: {number_of_rows}')

data_frame_tst = pd.read_csv(filepath)
len_df = len(data_frame_tst) # number of rows
#create NumPy array for column 'nr'
blocks = np.array(range(len_df), int)

#add 'nr' array as new column in DataFrame
data_frame_tst['nr'] = blocks.tolist()

filepath = "train_features1.csv"
print(f'\nPožymių masyvas iš:  {filepath}')
# print()
# print(data_frame.head(20))
data_frame_tst.to_csv(filepath, index=False, header=True)

# print()
# data_frame.info()

print()
# data_frame_tst = data_frame_tst[selected_features]
# print(data_frame_tst.head(20)) 


#        # Sukūriame tf.data.dataset klasės objektą duomenų skaitymui partijomis iš csv failo

# filepath = Path(features_dir, feature_set_fname)
filepath = "train_features1.csv"
print()
print(f'\nPožymių masyvas iš:  {filepath}')
batch_size = 5

selected_columns=['nr','RRl/RRr', '0']

dt_train_raw = tf.data.experimental.make_csv_dataset(
    filepath,
    batch_size=batch_size, # Artificially small to make examples easier to show.
    select_columns=selected_columns,
    label_name='nr',
    num_epochs=1,
    ignore_errors=True,)

# Kontrolinis skaitymas
# for batch, label in dt_train_raw.take(2):
#     # for key, value in batch.items():
#     #   print(f"{key:20s}: {value}")
#     print()
#     print(f"{'y':20s}: {label}")  

# Here's a simple function that will pack together all the columns:
def pack(features, label):
    return tf.stack(list(features.values()), axis=-1), label

# Apply this to each element of the dataset:
dt_train = dt_train_raw.map(pack)


#         Kontrolinis skaitymas stebint 'nr' pasiskirstymą partijose

for features, labels in dt_train.take(3):
    print()
    print(features.numpy())
    print()
    print(labels.numpy())


# file = object.myfilePath
# fileObject = csv.reader(file)
# row_count = sum(1 for row in fileObject)  # fileObject is your csv.reader

In [ ]:
# Apskaičiuojame MEAN ir STD iš atsitiktinių partijų. Skaičiuojama 
# atsitiktiniu būdu generuojant partijas. Ar pereinama per visas
# eilutes, priklauso nuo esančio take(N) skaičiaus

print(f'\nMEAN ir STD skaičiavimas naudojant tf.data.dataset')
take_N = 12
length = len(selected_columns)

rate = 100*batch_size*take_N/number_of_rows
print(f'\nnumber_of_rows: {number_of_rows} take_N: {take_N} batch_size: {batch_size}  rate: {rate:.1f}%'  )

np_features_set = np.empty((0,length-1),float)
print(np_features_set, np_features_set.shape)

for features, labels in dt_train.take(take_N):
    block = features.numpy()
    print()
    print(block)
    print()
    print(labels.numpy())

    np_features_set= np.append(np_features_set, block, axis=0)
    print(np_features_set, np_features_set.shape)

MEAN = np.mean(np_features_set, axis=0)
STD = np.std(np_features_set, axis=0)
print(f'\nMEAN: {MEAN}')
print(f'STD: {STD}')



In [ ]:
# MEAN ir STD iš numpy skaičiavimo bandymai

new1 = np.array([1.,2.,3.])
new1 = new1.reshape(1,3)
print(new1, new1.shape)
new2 = np.array([4.,5.,7.])
new2 = new2.reshape(1,3)
print(new2, new2.shape)
new3 = np.array([6.,7.,8.])
new3 = new3.reshape(1,3)
print(new3, new3.shape)
new4 = np.array([4.,5.,7.])
new4 = new4.reshape(1,3)
print(new4, new2.shape)
# print(np.vstack((new1,new2)))
# new = np.vstack((new1,new2))
# print(new)

# bandymas
np_features_set = np.empty((0,3),float)
# np_features_set = np.array([])
print(np_features_set, np_features_set.shape)

# np_features_set = np.vstack((np_features_set, new1))
np_features_set= np.append(np_features_set, new1, axis=0)
print(np_features_set, np_features_set.shape)

# np_features_set= np.vstack(np_features_set, new2)
np_features_set= np.append(np_features_set, new2, axis=0)
print(np_features_set, np_features_set.shape)

np_features_set= np.append(np_features_set, new3, axis=0)
print(np_features_set, np_features_set.shape)

np_features_set= np.append(np_features_set, new4, axis=0)
print(np_features_set, np_features_set.shape)

MEAN = np.mean(np_features_set, axis=0)
STD = np.std(np_features_set, axis=0)
# print(f'MEAN: {MEAN}')
print(f'MEAN: {MEAN}  STD: {STD}')